## Imports and Initialization

In [ ]:
# enable interactive components
%matplotlib widget

# Import packages
import numpy as np                # numerical routines
import matplotlib.pyplot as plt   # plotting
import ipywidgets as widgets      # provides interactive jupyter notebook components

In [ ]:
# ---- Store useful constants and variables

YR = 31557600.0        # year [sec]
PC = 3.08568e+18       # parsec [cm]
MSOL = 1.98841e+33     # solar Mass [g]
SPLC = 29979245800.0   # speed of light [cm/s]
NWTG = 6.6743e-08      # Newton's gravitational constant [cm^3/g/s^2]

MPC = PC * 1e6
GYR = YR * 1e9

In [ ]:
# ---- Set default cosmological parameters (using values from WMAP9)

OMEGA_R = 5.1426e-05                                # radiation
OMEGA_M = 0.2865                                    # matter
OMEGA_K = 0.0                                       # curvature
OMEGA_0 = (OMEGA_R + OMEGA_M + OMEGA_K)             # total density of positive-pressure components
OMEGA_L = 1.0 - OMEGA_0                             # dark energy; enforce total density is critical

DE_EOS = -1.0                                       # DE equation of state "w" in class
HUBBLE_0 = 2.2465e-18                               # 'H0' Hubble parameter (z=0) in units of [1/sec]
RHO_CRIT_0 = 3 * HUBBLE_0**2 / (8 * np.pi * NWTG)   # critical density of the universe [g/cm^3]

print(f"H_0 = {HUBBLE_0*MPC/1e5:.2f} [km/s/Mpc]")

## Define Cosmological Functions

In [ ]:
def efunc(redz, orad, omat, ok, ol, de_eos):
    """Calculate the ``E(z)`` function, s.t. the Hubble parameter is  ``H(z) = H_0 * E(z)``
    """
    zp1 = redz + 1.0
    
    # Calculate density parameters as function of scale-factor i.e. Omega_i(a)
    rad = orad * zp1 ** 4              # radiation
    mat = omat * zp1 ** 3              # matter
    cur = ok * zp1 ** 2                # curvature
    lam = ol * zp1 ** (3*(1+de_eos))   # dark energy
    
    # E(a) = sqrt(Omega_rad(a) + Omega_mat(a) + Omega_k(a) + Omega_Lambda(a))
    ef = rad + mat + cur + lam
    ef = np.sqrt(ef)
    return ef


def a_to_z(scafa):
    """Convert from scale-factor to redshift
    """
    return (1.0 / scafa) - 1.0


def z_to_a(redz):
    """Convert from redshift to scale-factor
    """
    return 1.0 / (1.0 + redz)


def Hz_func(redz, orad, omat, ok, ol, de_eos, H0):
    """Calculate the Hubble parameter as a function of redshift (and cosmological parameters)
    """    
    return H0 * efunc(redz, orad, omat, ok, ol, de_eos)


def dtdz_func(redz, orad, omat, ok, ol, de_eos, H0):
    """Calculate the derivative of time withe respect to redshift (dt/dz)
    """    
    Hz = Hz_func(redz, orad, omat, ok, ol, de_eos, H0)
    dtdz = 1.0 / ((1.0 + redz) * Hz)
    return dtdz


def time(redz, orad, omat, ok, ol, de_eos, H0):
    """Calculate the age of the universe as a function of redshift (and cosmological parameters)

    An array of redshifts (`zz`) are given, at which (dt/dz) is evaluated
    and integrated to get an array of ages of the universes (`tage`) at each redshift.
    i.e.
    
    t(z) = Integral_0^z (dt/dz') dz'
    
    Thus t(z) is evaluated at particular steps, t_i = t(z_i).  The smaller the spacing of `z_i`
    values (i.e. the `redz` input array), the more accurate the integral will be.

    Arguments
    ---------
    redz : array of scalar values,
        Redshifts at which to determine the age of the universe.
    orad : scalar,
        Omega_rad    the cosmological density parameter for radiation   at redshift zero.
    omat : scalar,
        Omega_mat    the cosmological density parameter for matter      at redshift zero.
    ok : scalar,
        Omega_k      the cosmological density parameter for curvature   at redshift zero.
    ol : scalar,
        Omega_Lambda the cosmological density parameter for dark energy at redshift zero.
    de_eos : scalar,
        The dark-energy equation-of-state parameter "w".
    H0 : scalar,
        Hubble constant at redshift zero (in units of [1/sec]).
    
    Returns
    -------
    tage : array or scalar,
        Age of the universe evaluated at redshifts `redz`, with length of redz minus one.
        In units of [sec].
    
    """
    
    # Evaluate the derivative (dt/dz) at the given redshifts
    dtdz = dtdz_func(redz, orad, omat, ok, ol, de_eos, H0)
    # Find the spacing between `redz` values (i.e. 'dz' = delta z)
    dz = np.diff(redz)

    # Integrate using the trapezoid rule
    # see: https://en.wikipedia.org/wiki/Trapezoidal_rule
    left = dtdz[:-1] * dz
    right = dtdz[1:] * dz
    # Calculate the time-interval integrated over each bin
    # i.e. time[4] is integral from redz[4] to redz[5]; note that `time` is now one shorter than `redz`
    tage = 0.5 * (left + right)
    
    # Perform a cumulative sum, such that time[4] is the integral from redz[0] to redz[4]
    tage = np.cumsum(tage)
    # The redshift values are assumed to be increasing, but we want to the age of the universe
    # calculated from z => infinity, so reverse the values
    tage = tage[-1] - tage
    
    return tage


def get_label(orad, omat, ok, ol, de_eos):
    """Get a formatted string printing the current cosmological parameters (for plotting purposes).
    """
    label = (
        f"$\Omega_r = {orad:.1e} \;\; \Omega_m = {omat:.2f} \;\; "
        f"\Omega_\Lambda = {ol:.2f} (w={de_eos:.1f}) \;\; \Omega_k = {ok:.1e}$"
    )
    return label

# Fig.1 – Cosmological Scale-Factor

In [ ]:
# ---- Basic plotting parameters

ZMIN = 1e-2
ZMAX = 1e4
XSCALE = 'log'
YSCALE = 'log'

# ---- Construct a figure, and setup the axes

# close previous figures, to avoid too many open ones after cells are re-run
plt.close('all')
fig, ax = plt.subplots(figsize=[8, 4])
# set axes parameters
ax.set(
    xscale=XSCALE, xlabel='redshift (z)', xlim=[ZMIN, ZMAX/2],
    yscale=YSCALE, ylabel='Age of the Universe [Gyr]', ylim=[1e-4, 3e1]
)
# add grid-lines
ax.grid(True, alpha=0.2)
# get label, and add to plot with current cosmological parameters
label = get_label(OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS)
ax.set_title(label)

# ---- Construct arrays to be plotted

# construct an array of redshift values with log-spacing 
redz = np.logspace(*np.log10([ZMIN, ZMAX]), 10000)
# start with redshift zero
redz = np.concatenate([[0.0], redz])
# integrate across redshifts to get age of universe
tt = time(redz, OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS, HUBBLE_0)
# convert [sec] ==> [Gyr]
tt = tt / GYR
# the integration returns one-less element than given, so select one-less redshift for plotting
zz = redz[1:]

# ---- Plot Age(z)

# Draw a (black dashed) line with standard cosmological parameters
ax.plot(zz, tt, 'k--', alpha=0.5, label=label)
# Also draw an orange line that will be updated as the parameters are changed
line, = ax.plot(zz, tt, color='darkorange', lw=2.0, alpha=0.8)

# ---- define a function to update the plot as the interactive parameters are changed

def update(omega_r=OMEGA_R, omega_m=OMEGA_M, omega_l=OMEGA_L, deos=DE_EOS):
    # calculate curvature to be consistent with input parameters
    omk = 1.0 - (omega_r + omega_m + omega_l)
    # calculate a new time evolution, convert to Gyr, update the plotted line
    tt = time(redz, omega_r, omega_m, omk, omega_l, deos, HUBBLE_0) / GYR
    line.set_ydata(tt)
    # get an updated plot label, add it to plot
    label = get_label(omega_r, omega_m, omk, omega_l, deos)
    ax.set_title(label)
    # redraw the plot
    fig.canvas.draw_idle()
    return

# ---- Construct interactive "widgets" to modify plotting parameters

w_omega_r = widgets.FloatLogSlider(OMEGA_R, min=-6.0, max=0.0, description='$\Omega_r$', readout_format='.2e')
w_omega_m = widgets.FloatSlider(OMEGA_M, min=-1.0, max=2.0, description='$\Omega_m$')
w_omega_l = widgets.FloatSlider(OMEGA_L, min=-1.0, max=2.0, description='$\Omega_\Lambda$')
w_omega_w = widgets.FloatSlider(DE_EOS, min=-4.0, max=2.0, description='$w$')
# Add interactive widgets
widgets.interact(update, omega_r=w_omega_r, omega_m=w_omega_m, omega_l=w_omega_l, deos=w_omega_w);

# add legend
plt.legend(loc='lower center')
# show plot
plt.show()

# Fig.2 – Cosmological Densities

In [ ]:
# ---- define basic plotting parameters

ZMIN = 1e-2
ZMAX = 1e4
XSCALE = 'log'
YSCALE = 'log'

# ---- Construct a figure, and setup the axes

# close previous figures, to avoid too many open ones after cells are re-run
plt.close('all')
fig, ax = plt.subplots(figsize=[8, 4])
# set axes parameters
ax.set(
    xscale='log', xlabel='Age of the Universe [Gyr]',
    yscale='log', ylabel='Cosmic Density', ylim=[1e-4, 1e13]
)
# add grid-lines
ax.grid(True, alpha=0.2)
# get label, and add to plot with current cosmological parameters
label = get_label(OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS)
ax.set_title(label)


# ---- Construct arrays to be plotted

# construct an array of redshift values with log-spacing 
redz = np.logspace(*np.log10([ZMIN, ZMAX]), 10000)
# start with redshift zero
redz = np.concatenate([[0.0], redz])
# integrate across redshifts to get age of universe, conver to Gyr
tt = time(redz, OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS, HUBBLE_0) / GYR
# the integration returns one-less element than given, so select one-less redshift for plotting
zz = redz[1:]

# Calculate the hubble parameters H(z) as a function of redshift
hub = Hz_func(zz, OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS, HUBBLE_0)

# Calculate the different densities of the universe, as a function of redshift
rho_crit = RHO_CRIT_0 * np.square(hub/HUBBLE_0)
rho_rad = OMEGA_R * (1.0 + zz) ** 4
rho_mat = OMEGA_M * (1.0 + zz) ** 3
norm = 1.0

ax.plot(tt, rho_rad * norm, 'r--', alpha=0.5)
ax.plot(tt, rho_mat * norm, 'b--', alpha=0.5)
rad_line, = ax.plot(tt, rho_rad * norm, 'r-', label='radiation')
mat_line, = ax.plot(tt, rho_mat * norm, 'b-', label='matter')
lam = OMEGA_L * norm * np.ones_like(tt)

ax.plot(tt, lam, 'k--', alpha=0.5)
lam_line, = ax.plot(tt, lam, 'k-', alpha=0.5, label='dark energy')

# ---- define a function to update the plot as the interactive parameters are changed

def update(omega_r=OMEGA_R, omega_m=OMEGA_M, omega_l=OMEGA_L, deos=DE_EOS):
    # calculate curvature to be consistent with input parameters
    omk = 1.0 - (omega_r + omega_m + omega_l)
    # calculate a new time evolution, convert to Gyr, update the plotted line
    tt = time(redz, omega_r, omega_m, omk, omega_l, deos, HUBBLE_0) / GYR
    rad_line.set_xdata(tt)
    mat_line.set_xdata(tt)
    lam_line.set_ydata(omega_l * norm * np.ones_like(tt))
    # get an updated plot label, add it to plot
    label = get_label(omega_r, omega_m, omk, omega_l, deos)
    ax.set_title(label)
    # redraw the plot
    fig.canvas.draw_idle()
    return
    
# ---- Construct interactive "widgets" to modify plotting parameters

w_omega_r = widgets.FloatLogSlider(OMEGA_R, min=-6.0, max=0.0, description='$\Omega_r$', readout_format='.2e')
w_omega_m = widgets.FloatSlider(OMEGA_M, min=-1.0, max=2.0, description='$\Omega_m$')
w_omega_l = widgets.FloatSlider(OMEGA_L, min=-1.0, max=2.0, description='$\Omega_\Lambda$')
w_omega_w = widgets.FloatSlider(DE_EOS, min=-4.0, max=2.0, description='$w$')
# Add interactive widgets
widgets.interact(update, omega_r=w_omega_r, omega_m=w_omega_m, omega_l=w_omega_l, deos=w_omega_w);

# add legend
plt.legend()
# show plot
plt.show()

# Fig.3 - Astron 429 - Distances

In [ ]:
def distance(redz, orad, omat, ok, ol, de_eos, H0):
    """Calculate the comoving distance as a function of redshift (and cosmological parameters)

    Arguments
    ---------
    redz : array_like (array or list) or scalar value,
        Redshifts at which to determine the age of the universe.
    orad : scalar,
        Omega_rad    the cosmological density parameter for radiation   at redshift zero.
    omat : scalar,
        Omega_mat    the cosmological density parameter for matter      at redshift zero.
    ok : scalar,
        Omega_k      the cosmological density parameter for curvature   at redshift zero.
    ol : scalar,
        Omega_Lambda the cosmological density parameter for dark energy at redshift zero.
    de_eos : scalar,
        The dark-energy equation-of-state parameter "w".
    H0 : scalar,
        Hubble constant at redshift zero (in units of [1/sec]).
    
    Returns
    -------
    dist : array of scalar,
        Comoving distance evaluated at redshifts `redz`.
        In units of [cm].
    
    """

    # ----
    # Fill in the body of this function so that it returns the comoving distance for each input
    # scale-factor.  You may use the `time()` function as a template, or you may write it in
    # your own way.  If you are familiar (or interested) in existing numerical quadrature functions
    # (such as `scipy.integrate.quadrature`), you are welcome to use those methods instead.
    # ----
    
    # ---- FILL IN YOUR CODE HERE! ----
    
    return dist
    

In [ ]:
# ---- define basic plotting parameters

ZMIN = 1e-2
ZMAX = 1e4
XSCALE = 'log'
YSCALE = 'log'

# ---- Construct a figure, and setup the axes

# close previous figures, to avoid too many open ones after cells are re-run
plt.close('all')
fig, ax = plt.subplots(figsize=[8, 4])
# set axes parameters
ax.set(
    xscale=XSCALE, xlabel='redshift (z)', xlim=[ZMIN, ZMAX/2],
    yscale=YSCALE, ylabel='Distance Measure [Mpc]' 
)
# add grid-lines
ax.grid(True, alpha=0.2)
# get label, and add to plot with current cosmological parameters
label = get_label(OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS)
ax.set_title(label)


# ---- Construct arrays to be plotted

# construct an array of redshift values with log-spacing 
redz = np.logspace(*np.log10([ZMIN, ZMAX]), 10000)
redz = np.concatenate([[0.0], redz])
# integrate across redshifts to get distances, convert to Mpc
dc = distance(redz, OMEGA_R, OMEGA_M, OMEGA_K, OMEGA_L, DE_EOS, HUBBLE_0) / MPC
zz = redz[1:]
# calculate angular-diameter distance (da) and luminosity distance (dl)
da = dc / (1.0 + zz)
dl = dc * (1.0 + zz)

# ---- Draw lines for each distance measure

# prep some labels
labels = ['$d_L$', '$d_c$', '$d_a$']
# initialize storage for lines
lines = [None] * 3
# We're going to draw two lines for each, dashed will stay fixed, solid lines will be updatable
# Iterate over two line-styles (ls)
for ls in ['--', '-']:
    # Iterate over each distance measure
    for ii, (line, lab, dist) in enumerate(zip(lines, labels, [dl, dc, da])):
        # choose a line color - use the same color as last loop, if this is the second time through
        color = ax._get_lines.get_next_color() if (line is None) else line.get_color()
        # label only the solid lines
        lab = None if (line is None) else lab
        # draw the line for this distance, color, line-style, label combination
        line, = ax.plot(zz, dist, lw=2.0, alpha=0.8, color=color, ls=ls, label=lab)
        # store the line for modification later
        lines[ii] = line
        
# ---- define a function to update the plot as the interactive parameters are changed

def update(omega_r=OMEGA_R, omega_m=OMEGA_M, omega_l=OMEGA_L, deos=DE_EOS):
    # calculate curvature to be consistent with input parameters
    omk = 1.0 - (omega_r + omega_m + omega_l)
    # calculate an updated comoving distance, convert to Myr
    dc = distance(redz, omega_r, omega_m, omk, omega_l, deos, HUBBLE_0) / MPC
    # calculate the other two distance measures
    da = dc / (1.0 + zz)
    dl = dc * (1.0 + zz)
    # Update data for lines
    lines[0].set_ydata(dl)
    lines[1].set_ydata(dc)
    lines[2].set_ydata(da)
    # update label
    label = get_label(OMEGA_R, OMEGA_M, omk, OMEGA_L, DE_EOS)
    ax.set_title(label)
    # redraw figure
    fig.canvas.draw_idle()
    return
    
# ---- Construct interactive "widgets" to modify plotting parameters

w_omega_r = widgets.FloatLogSlider(OMEGA_R, min=-6.0, max=0.0, description='$\Omega_r$', readout_format='.2e')
w_omega_m = widgets.FloatSlider(OMEGA_M, min=-1.0, max=2.0, description='$\Omega_m$')
w_omega_l = widgets.FloatSlider(OMEGA_L, min=-1.0, max=2.0, description='$\Omega_\Lambda$')
w_omega_w = widgets.FloatSlider(DE_EOS, min=-4.0, max=2.0, description='$w$')
# Add interactive widgets
widgets.interact(update, omega_r=w_omega_r, omega_m=w_omega_m, omega_l=w_omega_l, deos=w_omega_w);

# add legend
plt.legend(loc='lower center')
# show plot
plt.show()